In [2]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:100% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:14pt;}
div.output {font-size:14pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:14pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:14pt;padding:5px;}
table.dataframe{font-size:14px;}
</style>
"""))

# 1. 데이터 셋 가져오기

In [4]:
import pandas as pd
df = pd.read_csv('../01_python/data/ch13_df(결측치중위값으로대체).csv')
df.head()

,지역명,연도월,평당분양가격,연도,월
0,서울,2013년12월,18189.0,2013,12
1,부산,2013년12월,8111.0,2013,12
2,대구,2013년12월,8080.0,2013,12
3,인천,2013년12월,10204.0,2013,12
4,광주,2013년12월,6098.0,2013,12


- 지역명2 : 지역명필드를 라벨인코딩하여 추가(LabelIncoder)
- 독립변수 : 지역명2, 연도, 월 (x)
- 타겟변수 : 평당분양가격      (y)
- 독립변수와 타겟변수의 스케일 조정
    * 정규화(MinMaxScaler) 작업후 : 지역명2m, 연도m, 월m
    * 표준화(StandardScaler) 작업후 : 지역명2s, 연도s, 월s
  => 지역명, 연도, 월, 지역명2, 지역명2m, 연도m, 월m, 지역명2s, 연도s, 월s

# 2. 지역명의 라벨 인코딩
- 지역명2

# 3. MinMaxScale 조정

In [6]:
X = df[['지역명2','연도','월']].values  # numpy배열로 추출한 독립변수
y = df[['평당분양가격']].values
X.shape, y.shape

((2176, 2), (2176, 1))

# 4. standardScale 조정